# Stacked Timeseries Viewer Workflow



<div style="text-align: center;">
    <img src="./assets/231024_StackedTimeseries.png" alt="StackedTimeseries preview" width="450"/>
</div>

## Summary

This development workflow is intended to demonstrate the visualization of a set of stacked timeseries with HoloViz and Bokeh tools.

To launch this notebook's visualization as a standalone application, use `panel serve 0-StackedTimeseries.ipynb --show` at the command line.

<div class="admonition alert alert-info">
    <p class="admonition-title" style="font-weight:bold">Dependencies</p>
    <p>This workflow requires the <a href="./environment.yml">environment</a> specified in this workflow directory.</p>
</div>


In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np; np.random.seed(0)
import pandas as pd
from scipy.stats import zscore
import string

# Viz
import colorcet as cc
import holoviews as hv; hv.extension('bokeh')
from holoviews.plotting.links import RangeToolLink
# from holoviews.operation.datashader import rasterize
from holoviews import Dataset
from bokeh.models import HoverTool, WheelZoomTool
import panel as pn; pn.extension(template='bootstrap')
from holonote.annotate import Annotator

## Generate sine wave data

In [ ]:
n_channels = 25
n_seconds = 15
fs = 250  # Sampling frequency

init_freq = 1  # Initial sine wave frequency in Hz
freq_inc = 20/n_channels  # Frequency increment
amplitude = 1

total_samples = n_seconds * fs
time = np.linspace(0, n_seconds, total_samples)
channels = [f'EEG {i}' for i in range(n_channels)]

data = np.array([amplitude * np.sin(2 * np.pi * (init_freq + i * freq_inc) * time)
                 for i in range(n_channels)])
print(f'shape: {data.shape} (n_channels, samples) ')

## Generate time-range annotations

In [ ]:
def create_range_annotations(n_total_seconds: int, n_categories: int, 
                             n_total_annotations: int, duration: int = 1) -> pd.DataFrame:

    
    start_times = np.sort(np.random.randint(0, n_total_seconds - duration, n_total_annotations))
    
    # Ensure the annotations are non-overlapping
    for i in range(1, len(start_times)):
        if start_times[i] < start_times[i-1] + duration:
            start_times[i] = start_times[i-1] + duration
    end_times = start_times + duration
    categories = np.random.choice(list(string.ascii_uppercase)[:n_categories], n_total_annotations)
    
    df = pd.DataFrame({
        'start': start_times,
        'end': end_times,
        'category': categories
    })
    df['category'] = df['category'].astype('category')
    return df

np.random.seed(5)
n_categories = 3
n_total_annotations = 5
annotations_df = create_range_annotations(n_seconds, n_categories, n_total_annotations)
annotations_df.sample(5)


## Visualize stacked timeseries

In [ ]:
xzoom_out_extent = 2
start_t_disp = 4.5 # start time of initially displayed window 
max_t_disp = xzoom_out_extent # max time in seconds to initially display
max_ch_disp = 20  # max channels to initially display
max_y_disp = np.min((max_ch_disp - 1.5, n_channels - 1.5))
subcoord_btm = -0.5 # auto lower xlim of first subcoord
clim_mul = 1 # color limit multiplier.. adjusts the levels on the minimap

annotator = Annotator({"Time": float}, fields=["category"])

if annotator.df.empty:
    annotations_df = create_range_annotations(n_seconds, n_categories, n_total_annotations)
    annotator.define_annotations(annotations_df, Time=("start", "end"))

annotations_overlay = annotator.get_element("Time")

hover = HoverTool(tooltips=[
    ("Channel", "@channel"),
    ("Time", "$x s"),
    ("Amplitude", "$y µV")])

channel_curves = []
for channel, channel_data in zip(channels, data):
    ds = Dataset((time, channel_data, channel), ["Time", "Amplitude", "channel"])
    curve = hv.Curve(ds, "Time", ["Amplitude", "channel"], label=f'{channel}')
    curve.opts(color="black", line_width=1, subcoordinate_y=True, tools=[hover])
    channel_curves.append(curve)

curves = hv.Overlay(channel_curves, kdims="Channel")
eeg_viewer = (annotations_overlay * curves)
eeg_viewer = eeg_viewer.opts(
    xlabel="Time (s)", ylabel="Channel", show_legend=False,
    padding=0, aspect=1.5, responsive=True, shared_axes=False, framewise=False,
     #ylim does not work with subcoordinate_y
    # xlim=(start_t_disp, start_t_disp+max_t_disp), ylim=(subcoord_btm, subcoord_btm+max_y_disp),
    backend_opts={
        "y_range.start": subcoord_btm, # required as long as ylim doesn't work
        "y_range.end": subcoord_btm + max_y_disp, # required as long as ylim doesn't work
        "x_range.start": start_t_disp,
        "x_range.end": start_t_disp + max_t_disp,
        "x_range.bounds": (time.min(), time.max()), # absolute outer limits on pan/zoom
        "y_range.bounds": (0, len(channels)),
        "x_range.max_interval": xzoom_out_extent
    })

y_positions = range(len(channels))
yticks = [(i, ich) for i, ich in enumerate(channels)]
z_data = zscore(data, axis=1)
# Does not currently work with rasterize on the minimap image.
minimap = hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)")
minimap = minimap.opts(
    cmap="RdBu_r", colorbar=False, xlabel='',
    alpha=.3, yticks=[yticks[0], yticks[-1]],
    toolbar='disable', # needed to prevent zoom and pan behavior on image
    height=120, responsive=True, default_tools=[],
    clim=(-z_data.std()*clim_mul, z_data.std()*clim_mul))

RangeToolLink(minimap, curves, axes=["x", "y"],  # Hoxbro: Note I use curve here
              boundsx=(start_t_disp, start_t_disp + max_t_disp), #required for reset behavior
              boundsy=(subcoord_btm, subcoord_btm + max_y_disp) #required for reset behavior
             )

eeg_app = pn.Column((eeg_viewer + minimap * annotations_overlay).cols(1), min_height=650).servable()

In [ ]:
# Setup Annotator styling and groupby
unique_categories = ["A", "B", "C"]
color_map = dict(zip(unique_categories, cc.glasbey[:len(unique_categories)]))

annotator.style.color = hv.dim("category").categorize(categories=color_map, default="grey")
annotator.groupby = "category"
widget = pn.widgets.MultiSelect(name="Show category", value=["B", "C"], options=["A", "B", "C"], )
annotator.visible = widget
widget.servable(location='sidebar')

# Setup app component
from holonote.app import PanelWidgets

annotator_tools = PanelWidgets(annotator, {"category": unique_categories})
annotator_tools_pn = pn.panel(annotator_tools).servable(target='sidebar')

pn.Column(widget, annotator_tools_pn)

In [ ]:
eeg_app